In [1]:
import sys
import os
import math
import random
import heapq 
import time
import copy
import gc
import numpy as np
import pandas as pd
from functools import reduce
from scipy.spatial.distance import pdist
from PIL import Image
import matplotlib.pyplot as plt
import cv2
#import faiss
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.cuda.set_device(5)
print (torch.cuda.current_device())

5


In [2]:
#1. Read data with List storage Name:[name],I:[img],Y[type]
root_dir = '/data/fjsdata/fundus/kaggle_DR/train/' #the path of images
trainset = pd.read_csv("/data/fjsdata/fundus/kaggle_DR/CBIR_train.csv" , sep=',')#load dataset
testset = pd.read_csv("/data/fjsdata/fundus/kaggle_DR/CBIR_test.csv" , sep=',')#load dataset
tstart = time.time()
#read train image with CV
trN, trI, trY = [],[],[]
for iname, itype in np.array(trainset).tolist():
    try:
        image_path = os.path.join(root_dir, iname+'.jpeg')
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        trN.append(iname)
        trI.append(img)
        trY.append(itype)  
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(trN),trainset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(trN))
#read test image with CV
teN, teI, teY = [],[],[]
for iname, itype in np.array(testset).tolist():
    try:
        image_path = os.path.join(root_dir, iname+'.jpeg')
        img = cv2.resize(cv2.imread(image_path).astype(np.float32), (512, 512))#(1024,1024,3)->(512,512,3)
        teN.append(iname)
        teI.append(img)
        teY.append(itype)  
    except:
        print(iname+":"+str(image_path))
    sys.stdout.write('\r{} / {} '.format(len(teN),testset.shape[0]))
    sys.stdout.flush()
print('The length of train set is %d'%len(teN))
elapsed = time.time() - tstart    
print('Completed buliding index in %d seconds' % int(elapsed))

#Generate image pairs for model
def onlineGenImgPairs(spls=len(trY)):
    idx_sf = random.sample(range(0, len(trY)),spls)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf
trI1_sf, trI2_sf, trY_sf = onlineGenImgPairs()

7200 / 7200 The length of train set is 7200
800 / 800 The length of train set is 800
Completed buliding index in 5298 seconds


In [4]:
#define Hashing network with pytorch
class HNet(nn.Module): 
    def __init__(self,inChannels=3, outHashcode=16):
        super(HNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,512,512)->(8,256,256)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(8,256,256)->(8,128,128)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(8)
        #layer: spatial attention, pass
        #layer3: Convolution, (8,128,128)->(2,64,64)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(2)
        self.relu2 = nn.ReLU(inplace=True)
        #layer4: mean pooling, (2,64,64)->(2,32,32)
        self.avgpool = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(2)
        #layer5: fully connected, 2*32*32->512
        self.fcl1 = nn.Linear(2*32*32,512)
        self.relu3 = nn.ReLU(inplace=True)
        #layer6: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,outHashcode)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer: attention,pass
        #layer3: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer4: mean pooling
        x = self.avgpool(x)
        x = self.bn4(x)
        #layer5:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu3(x)
        #layer6: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss

#define model
model = HNet(outHashcode=16).cuda()
criterion  = HashLossFunc(margin=0.5).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
num_batches = len(trY_sf) // batchSize
for epoch in range(10):#iteration
    losses = []
    for i in range(num_batches): 
        optimizer.zero_grad() #grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teY[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 360 / 360 : loss = 21.019512Eopch:     1 mean_loss = 18.395586
 360 / 360 : loss = 14.683266Eopch:     2 mean_loss = 18.607869
 360 / 360 : loss = 17.765585Eopch:     3 mean_loss = 18.950913
 360 / 360 : loss = 18.370186Eopch:     4 mean_loss = 18.709541
 360 / 360 : loss = 21.461706Eopch:     5 mean_loss = 19.821693
 360 / 360 : loss = 23.227335Eopch:     6 mean_loss = 19.510930
 360 / 360 : loss = 26.531111Eopch:     7 mean_loss = 21.059035
 360 / 360 : loss = 23.063339Eopch:     8 mean_loss = 21.173919
 360 / 360 : loss = 21.306534Eopch:     9 mean_loss = 20.333095
 360 / 360 : loss = 25.292913Eopch:    10 mean_loss = 20.897715
best_loss = 18.395586
 79 / 80 0 mHR@5=0.414750, mAP@5=0.410838, mRR@5=0.980220
mHR@10=0.412000, mAP@10=0.402982, mRR@10=0.944890
mHR@15=0.412833, mAP@15=0.399164, mRR@15=0.908331
mHR@20=0.408813, mAP@20=0.392245, mRR@20=0.884327


In [5]:
#define Attention-based Hashing network with pytorch
class SpatialAttention(nn.Module):#spatial attention layer
    def __init__(self):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
class AHNet(nn.Module): #deep Hashint Network:DHNet
    def __init__(self,inChannels=3,outHashcode=16):
        super(AHNet, self).__init__()
        #(channels, Height, Width)
        #layer1: Convolution, (3,512,512)->(8,256,256)
        self.conv1 = nn.Conv2d(in_channels=inChannels, out_channels=8, kernel_size=3, padding=1, stride=2)
        self.bn1 = nn.BatchNorm2d(8)
        self.relu1 = nn.ReLU(inplace=True)
        #layer2: max pooling,(8,256,256)->(8,128,128)
        self.maxpool = nn.MaxPool2d(kernel_size=3, padding=1, stride=2)
        self.bn2 = nn.BatchNorm2d(8)
        #layer3: Spatial Attention Layer, (8,256,256)->(8,256,256)
        self.sa = SpatialAttention()
        #layer4: Convolution, (8,128,128)->(2,64,64)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=2, kernel_size=3, padding=1, stride=2)
        self.bn3 = nn.BatchNorm2d(2)
        self.relu2 = nn.ReLU(inplace=True)
        #layer5: mean pooling, (2,64,64)->(2,32,32)
        self.avgpool = nn.AvgPool2d(kernel_size=3, padding=1, stride=2)
        self.bn4 = nn.BatchNorm2d(2)
        #layer6: fully connected, 2*32*32->512
        self.fcl1 = nn.Linear(2*32*32,512)
        self.relu3 = nn.ReLU(inplace=True)
        #layer7: Hashing layer, 512->16
        self.fcl2 = nn.Linear(512,outHashcode)#
        self.tanh = nn.Tanh() #{-1,1}
              
    def forward(self,x):
        #input: (batch_size, in_channels, Height, Width)
        #output: (batch_size, out_channels, Height, Width)
        #layer1: convolution
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        #layer2: max pooling
        x = self.maxpool(x)
        x = self.bn2(x)
        #layer3: Attention
        x = self.sa(x)*x
        #layer4: Convolution
        x = self.conv2(x)
        x = self.bn3(x)
        x = self.relu2(x)
        #layer5: mean pooling
        x = self.avgpool(x)
        x = self.bn4(x)
        #layer6:fully connected
        x = x.view(x.size(0),-1) #transfer three dims to one dim
        x = self.fcl1(x)
        x = self.relu3(x)
        #layer7: Hashing layer
        x = self.fcl2(x)
        x = self.tanh(x)
                
        return x
    
#https://pytorch-cn.readthedocs.io/zh/latest/    
#https://github.com/filipradenovic/cnnimageretrieval-pytorch/blob/master/cirtorch/layers/functional.py
class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
    
    def forward(self,h1,h2,y): 
        #h1=h2:NxD,y:N
        dim = h1.shape[1]
        euc_dist = F.pairwise_distance(h1, h2, p=2, eps=1e-06) # Calcualte Euclidean Distance
        sim_term = 0.5*(1-y)*euc_dist #penalize the similar iamge pairs when y=0
        unsim_term = 0.5*y*torch.clamp(self.margin*dim-euc_dist,0)#penalize the unsimlar image pairs when y =1
        reg_term = self.alpha * ( torch.sum((torch.abs(h1)-1),dim=1) + torch.sum((torch.abs(h2)-1),dim=1) ) #regularization term
        #loss = torch.mean(sim_term + unsim_term + reg_term) 
        loss = torch.sum(sim_term + unsim_term+ reg_term) 
        return loss
    
#define model
model = AHNet(outHashcode=16).cuda()
criterion  = HashLossFunc(margin=0.5).cuda() #define loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
#train model
best_net, best_loss = None, float('inf')
batchSize = 10
num_batches = len(trY_sf) // batchSize
for epoch in range(10):#iteration
    losses = []
    for i in range(num_batches):   
        optimizer.zero_grad() #grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trY_sf), (i+1)*batchSize])
        I1_batch = torch.from_numpy(trI1_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        I2_batch = torch.from_numpy(trI2_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Y_batch = torch.from_numpy(trY_sf[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        #forword
        X1_batch = model(I1_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        X2_batch = model(I2_batch.permute(0, 3, 1, 2))
        #binary-like loss
        loss = criterion(X1_batch,X2_batch,Y_batch)
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))

#release gpu memory
model = model.cpu()
torch.cuda.empty_cache()
#hash code of train data from model
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#hash code of test data from model
teF = []
num_batches = len(teI) // batchSize
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(teI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(teI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch = torch.sign(best_net(I_batch.permute(0, 3, 1, 2)))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    teF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
    
#train data with list: trData, trI, trF, trY
#test data with list: teData, teI, teF, teY
for topk in [5,10,15,20]:
    MHR = [] #mean Hit ratio 
    MAP = [] #mean average precision
    MRR = [] #mean reciprocal rank
    for i, teVal in enumerate(teF):
        stype = teY[i]
        map_item_score = {}
        for j, trVal in enumerate(trF):
            map_item_score[j] = pdist(np.vstack([teVal,trVal]),'hamming')
        ranklist = heapq.nsmallest(topk, map_item_score, key=map_item_score.get)
        #perfromance
        pos_len = 0
        rank_len = 0
        mrr_flag = 0
        for j in ranklist:
            dtype = trY[j]
            rank_len=rank_len+1
            if stype==dtype:  #hit
                MHR.append(1)
                pos_len = pos_len +1
                MAP.append(pos_len/rank_len) 
                if mrr_flag==0: 
                    MRR.append(pos_len/rank_len)
                    mrr_flag =1
            else: 
                MHR.append(0)
                MAP.append(0)   
    print("mHR@{}={:.6f}, mAP@{}={:.6f}, mRR@{}={:.6f}".format(topk,np.mean(MHR),topk,np.mean(MAP), topk, np.mean(MRR)))

 360 / 360 : loss = 20.115781Eopch:     1 mean_loss = 19.016349
 360 / 360 : loss = 24.471451Eopch:     2 mean_loss = 19.765195
 360 / 360 : loss = 22.322865Eopch:     3 mean_loss = 20.120964
 360 / 360 : loss = 22.085339Eopch:     4 mean_loss = 19.866186
 360 / 360 : loss = 22.620518Eopch:     5 mean_loss = 19.405966
 360 / 360 : loss = 20.184763Eopch:     6 mean_loss = 19.601254
 360 / 360 : loss = 16.736885Eopch:     7 mean_loss = 19.365557
 360 / 360 : loss = 19.977182Eopch:     8 mean_loss = 19.133452
 360 / 360 : loss = 18.430227Eopch:     9 mean_loss = 19.438465
 360 / 360 : loss = 20.327784Eopch:    10 mean_loss = 19.764795
best_loss = 19.016349
 79 / 80 0 mHR@5=0.419000, mAP@5=0.416462, mRR@5=0.983622
mHR@10=0.417875, mAP@10=0.413497, mRR@10=0.966485
mHR@15=0.418583, mAP@15=0.412259, mRR@15=0.954107
mHR@20=0.418438, mAP@20=0.411663, mRR@20=0.949105


In [9]:
#generate dataset
dataset = pd.read_csv("/data/fjsdata/fundus/kaggle_DR/trainLabels.csv" , sep=',')#load dataset
trainset, testset = [], []
#dataset = dataset[dataset['image'].contains('left')]
tf = dataset['image'].str.contains('right')#right eye
dataset = dataset[tf]
print (dataset.shape)
print (dataset['level'].value_counts())
ds =dataset[dataset['level']==0].sample(3376).sample(frac=1) #0:3376-338,shuffle
testset.extend(np.array(ds).tolist()[0:339])
trainset.extend(np.array(ds).tolist()[339:])
ds =dataset[dataset['level']==1].sample(frac=1)#1:1231-123,shuffle
testset.extend(np.array(ds).tolist()[0:123])
trainset.extend(np.array(ds).tolist()[123:])
ds =dataset[dataset['level']==2].sample(frac=1)#2:2590-259,shuffle
testset.extend(np.array(ds).tolist()[0:259])
trainset.extend(np.array(ds).tolist()[259:])
ds =dataset[dataset['level']==3].sample(frac=1)#3:448-44,shuffle
testset.extend(np.array(ds).tolist()[0:44])
trainset.extend(np.array(ds).tolist()[44:])
ds =dataset[dataset['level']==4].sample(frac=1)#4:355-35,shuffle
testset.extend(np.array(ds).tolist()[0:35])
trainset.extend(np.array(ds).tolist()[35:])
print (len(trainset))
print (len(testset))
pd.DataFrame(trainset).to_csv('/data/fjsdata/fundus/kaggle_DR/CBIR_train.csv',index=False)
pd.DataFrame(testset).to_csv('/data/fjsdata/fundus/kaggle_DR/CBIR_test.csv',index=False)
del ds,dataset
gc.collect() #release cpu memory

(17563, 2)
0    12939
2     2590
1     1231
3      448
4      355
Name: level, dtype: int64
7200
800


12